In [ ]:
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms.minimum_eigensolvers import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler

In [ ]:
installation_costs = {
    'Supermarket_1': 30000,
    'Church_1': 10000, 
    'Sport_center': 44000,
    'Supermarket_2': 40000,       
    'Shopping_center_2': 27000,
    'Shopping_center_1': 35000,
    'Cinema': 52000,
    'University_1': 65000,
    'Library': 22000,
    'Park_1': 18000,
    'Museum': 42000,
    'Train_station': 54000,
    'Airport': 97000,
    # 'Hospital': 72000,
    # 'Office_building': 55000
}

traffic_values = {
    'Supermarket_1': [0.06, 0.21, 0.86, 0.38, 0.16, 0.06, 0.38],
    'Church_1': [0.74, 0.74, 0.6, 0.45, 0.22, 0.08, 0.15],
    'Sport_center': [0.30, 0.5, 0.51, 0.50, 0.46, 0.06, 0.73],
    'Supermarket_2': [0.91, 0.24, 0.98, 0.70, 0.89, 0.42, 0.2],
    'Shopping_center_2': [0.54, 0.59, 0.11, 0.29, 0.15, 0, 0],
    'Shopping_center_1': [0.28, 0.84, 0.25, 0.07, 0.13, 0.31, 0.19],
    'Cinema': [0.40, 0.77, 0.99, 0.90, 0.84, 0.10, 0.58],
    'University_1': [0.03, 0.014, 0.86, 0.48, 0.71, 0.73, 0.99],
    'Library': [0.56, 0.96, 0.79, 0.14, 0.28, 0.09, 0.35],
    'Park_1': [0.82, 0.66, 0.42, 0.18 , 0.24, 0.72, 0.39],
    'Museum': [0.18, 0.49, 0.62, 0.37, 0.93, 0, 0],
    'Train_station': [0.27, 0.08, 0.73, 0.15, 0.81, 0.98, 0.39],
    'Airport': [0.43, 0.19, 0.023, 0.094, 0.84, 0.69, 0.12],
    # 'Hospital': [0.61, 0.023, 0.72, 0.25, 0.44 ,0.79, 0.59],
    # 'Office_building': [0.075, 0.34, 0.58, 0.72, 0.42, 0, 0]
}

budget = 120000 # 120k

In [ ]:
# Avg traffic values
traffic_values_list = [sum(l)/len(l) for l in traffic_values.values()]
installation_costs_list = list(installation_costs.values())
budget = int((budget / max(installation_costs_list))*100)

# Normalize
installation_costs_list = [int((x / max(installation_costs_list))*100) for x in installation_costs_list]
installation_costs_list

In [ ]:

prob = Knapsack(values=traffic_values_list, weights=installation_costs_list, max_weight=budget)
qp = prob.to_quadratic_program()
print(qp.prettyprint())

In [9]:
# QAOA
meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, sampler=Sampler(), optimizer=COBYLA()))
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))
print("\ntime:", result.min_eigen_solver_result.optimizer_time)

In [ ]:
from qiskit_optimization.converters import QuadraticProgramToQubo

# intermediate QUBO form of the optimization problem
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)
print(qubo.prettyprint())

In [ ]:
# qubit Hamiltonian and offset
op, offset = qubo.to_ising()
print(f"num qubits: {op.num_qubits}, offset: {offset}\n")
print(op)